In [108]:
%load_ext autoreload
%autoreload 2
import itertools as itt
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import glob
import multiprocessing as mp
import argparse
import re
import shutil
import mat73

from minian.utilities import open_minian

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
minian_cross = '/scratch/09117/xz6783/Xu/PL_miniscope/PL/Analysis/Cross_regist/Satiation_vs_conflict/minian_cross'

In [86]:
cell_reg = '/scratch/09117/xz6783/minian/cross_regist/mat_files/'
cell_reg_output = '/scratch/09117/xz6783/Xu/PL_miniscope/PL/Analysis/Cross_regist/Satiation_vs_conflict/cell_reg/'

In [6]:
conflict_path = "/scratch/09117/xz6783/minian/Cat_Odor_Conflict/Conflict/"
satiation_path = "/scratch/09117/xz6783/minian/Satiation/Session Combined"
id_path = "/scratch/09117/xz6783/minian/Cat_Odor_Conflict/Conflict/"
files = os.listdir(id_path)

animal_ids = [file for file in files if os.path.isdir(os.path.join(id_path, file)) and len(file) == 3 and file[0].isdigit()]

print(len(animal_ids))
animal_ids

14


['2Z4',
 '6S5',
 '9M6',
 '2I5',
 '7H6',
 '1S5',
 '7M6',
 '7B6',
 '8H6',
 '2N6',
 '1H6',
 '5B6',
 '6H6',
 '6M6']

In [62]:
for animal_id in animal_ids:
    # Get minian_ds for conflict
    dpath_conflict = os.path.join(conflict_path, animal_id)
    pattern_conflict = os.path.join(dpath_conflict, "**", "minian")
    minian_ds_path_conflict = glob.glob(pattern_conflict, recursive=True)[0]
    if not os.path.exists(minian_ds_path_conflict):
        print(f"Minian dataset path {minian_ds_path_conflict} not found for animal ID {animal_id}")

    minian_ds_conflict = open_minian(minian_ds_path_conflict)
    
    # Assuming labels and unitid are as you defined
    labels_conflict = minian_ds_conflict['C'].unit_labels.values
    unitid_conflict = minian_ds_conflict['C'].unit_id.values

    # Create the mapping directly from unit_id to label
    unit_id_to_label_conflict = {u_id: label for label, u_id in zip(labels_conflict, unitid_conflict)}
    
    # Get minian_ds for satiation
    dpath_satiation = os.path.join(satiation_path, animal_id)
    pattern_satiation = os.path.join(dpath_satiation, "**", "minian")
    minian_ds_path_satiation = glob.glob(pattern_satiation, recursive=True)[0]
    if not os.path.exists(minian_ds_path_satiation):
        print(f"Minian dataset path {minian_ds_path_satiation} not found for animal ID {animal_id}")

    minian_ds_satiation = open_minian(minian_ds_path_satiation)
    
    # Assuming labels and unitid are as you defined
    labels_satiation = minian_ds_satiation['C'].unit_labels.values
    unitid_satiation = minian_ds_satiation['C'].unit_id.values

    # Create the mapping directly from unit_id to label
    unit_id_to_label_satiation = {u_id: label for label, u_id in zip(labels_satiation, unitid_satiation)}
 
    # Get mapping pickle file
    pattern = os.path.join(minian_cross, f"{animal_id}*.pkl")
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        print(f"No mapping pickle file found for animal ID {animal_id}")
        continue  # Skip to the next animal_id if no pickle file found

    # Load the DataFrame from the first matching file
    pickle_file_path = matching_files[0]
    cross_map = pd.read_pickle(pickle_file_path).iloc[:, 1:3].dropna().astype('int64')
    
    # Update DataFrame with mapped labels
    cross_map['session']['Conflict'] = cross_map['session']['Conflict'].map(unit_id_to_label_conflict)
    cross_map['session']['Satiation'] = cross_map['session']['Satiation'].map(unit_id_to_label_satiation)

    # Drop duplicate rows where 'Conflict' and 'Satiation' are the same
    cross_map = cross_map.drop_duplicates(subset=[('session', 'Conflict'), ('session', 'Satiation')])

    # Save the modified DataFrame back to the original file
    cross_map.to_pickle(pickle_file_path)


In [96]:
f_pattern = r'^cellRegistered.*\.mat$'



for animal_id in os.listdir(cell_reg):
    path_animal = os.path.join(cell_reg, animal_id)
    glob_pattern = os.path.join(path_animal, "**", "*.mat")
    all_mat_files = glob.glob(glob_pattern, recursive=True)
    matching = [file for file in all_mat_files if re.match(f_pattern, os.path.basename(file))]
    if matching:  # Check if there are any matching files
        latest_file = sorted(matching)[-1]  # Get the latest file
        new_filename = f"{animal_id}_{os.path.basename(latest_file)}"  # Prepend animal_id to the filename
        new_file_path = os.path.join(cell_reg_output, new_filename)  # Define new file path
        shutil.copy2(latest_file, new_file_path)  # Copy the file with the new name

In [196]:
for animal_id in animal_ids:
    # Get minian_ds for conflict
    dpath_conflict = os.path.join(conflict_path, animal_id)
    pattern_conflict = os.path.join(dpath_conflict, "**", "minian")
    minian_ds_path_conflict = glob.glob(pattern_conflict, recursive=True)[0]
    if not os.path.exists(minian_ds_path_conflict):
        print(f"Minian dataset path {minian_ds_path_conflict} not found for animal ID {animal_id}")

    minian_ds_conflict = open_minian(minian_ds_path_conflict)

    # Assuming labels and unitid are as you defined
    labels_conflict = minian_ds_conflict['C'].unit_labels.values
    unitid_conflict = minian_ds_conflict['C'].unit_id.values

    # Create the mapping directly from unit_id to label
    unit_id_to_label_conflict = {u_id: label for label, u_id in zip(labels_conflict, unitid_conflict)}
    mat_to_unitid_conflict = {label: u_id for label, u_id in zip(np.arange(1, len(unitid_conflict)+1), unitid_conflict)}

    # Get minian_ds for satiation
    dpath_satiation = os.path.join(satiation_path, animal_id)
    pattern_satiation = os.path.join(dpath_satiation, "**", "minian")
    minian_ds_path_satiation = glob.glob(pattern_satiation, recursive=True)[0]
    if not os.path.exists(minian_ds_path_satiation):
        print(f"Minian dataset path {minian_ds_path_satiation} not found for animal ID {animal_id}")
        continue
    minian_ds_satiation = open_minian(minian_ds_path_satiation)

    # Assuming labels and unitid are as you defined
    labels_satiation = minian_ds_satiation['C'].unit_labels.values
    unitid_satiation = minian_ds_satiation['C'].unit_id.values

    # Create the mapping directly from unit_id to label
    unit_id_to_label_satiation = {u_id: label for label, u_id in zip(labels_satiation, unitid_satiation)}
    mat_to_unitid_satiation = {label: u_id for label, u_id in zip(np.arange(1, len(unitid_satiation)+1), unitid_satiation)}

    # Get mapping pickle file
    pattern = os.path.join(cell_reg_output, f"{animal_id}*.mat")
    matching_files = glob.glob(pattern)

    # Load the DataFrame from the first matching file
    mat_file_path = matching_files[0]
    cell_to_index_map = mat73.loadmat(os.path.join(cell_reg_output, mat_file_path))['cell_registered_struct']['cell_to_index_map'].astype('int64')

    # Matlab indexes starting from 1. Correct this for Python.
    # Then NaN out the unmatched cells.
    match_map = pd.DataFrame(cell_to_index_map)
    match_map.columns = ['Conflict_cell_id', 'Satiation_cell_id']

    # # Update DataFrame with mapped labels
    match_map['Conflict_cell_id'] = match_map['Conflict_cell_id'].map(mat_to_unitid_conflict)
    match_map['Satiation_cell_id'] = match_map['Satiation_cell_id'].map(mat_to_unitid_satiation)
    match_map['Conflict_cell_id'] = match_map['Conflict_cell_id'].map(unit_id_to_label_conflict)
    match_map['Satiation_cell_id'] = match_map['Satiation_cell_id'].map(unit_id_to_label_satiation)

    # # Drop duplicate rows where 'Conflict' and 'Satiation' are the same
    match_map = match_map.dropna().drop_duplicates(subset=['Conflict_cell_id', 'Satiation_cell_id'])
    pickle_file_path = matching_files[0].split('.')[0] + '.pkl'

    # Save the modified DataFrame back to the original file
    match_map.to_pickle(pickle_file_path)